<a href="https://colab.research.google.com/github/Shai2u/BatYamNY/blob/master/population_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
#Installing specific packages.
!pip install dash==2.0.0
!pip install jupyter_dash
!pip install geopandas
!pip install dash-leaflet==0.1.23
!pip install dash-extensions==0.0.65

In [74]:

from dash import Dash, html, dcc, Input, Output
from dash_extensions.javascript import  assign

import dash_leaflet.express as dlx
import plotly.express as px
import plotly.graph_objects as go
import dash_leaflet as dl
import pandas as pd
import numpy as np
import json
import geopandas as gpd
from jupyter_dash import JupyterDash
import datetime

In [75]:
#pandas configuration
pd.options.display.max_columns=100

# Loading Files

In [76]:

#Statistical Stats
statisticalStatsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/statistical_tract_4326.geojson')
statisticalStatsJson = json.loads(statisticalStatsGdf.to_json())
#building simulation
simulatedBldgsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/buildings_for_dashboard_4326.geojson')
simulatedBldgsJson = json.loads(simulatedBldgsGdf.to_json())
simulatedBldgsGdf['start_date'] = pd.to_datetime(simulatedBldgsGdf['start_date'])
simulatedBldgsGdf['end_date'] = pd.to_datetime(simulatedBldgsGdf['end_date'])
#agents track
agents_track_status = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/agents_track_status.csv')
agents_track_status.drop(columns='Unnamed: 0',inplace=True)

agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

year_ranges = agents_stat_summary_by_year['year']

attribution = '© OpenStreetMap contributors, © CARTO'
cartoUrl = 'http://basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'

# style =   color: 'red',
#                 weight: 10,
#                 opacity: .7,
#                 dashArray: '20,15',
#                 lineJoin: 'round'


## Pre-processing

In [285]:

agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']


In [286]:
# Generate count status graph
def get_status_graph(bldg_status):
  #"status"
  fig = px.bar(bldg_status, x='status', y="count",width=500,height=200,template='plotly_white',title="Current Building Status",category_orders={'status':['Building before', 'Construction', 'Building after']})
  fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
  showlegend=True,

  # font=dict(
  #     size=15,
  # )
  )
  fig.update_yaxes(automargin=True)
  fig.update_yaxes(range=[0, 70])
  return fig


# Dot Matrix

In [287]:

    #fig.write_image(f'matrixPlot_{ticNumber}.png')

#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])

list_ = ['Staying']* agents_time_new_Comers_stay.loc[20,'stay'].astype(int)
list_ = list_ + ['New Comers']*agents_time_new_Comers_stay.loc[20,'New Comers'].astype(int)
list_ = list_ + ['Future Units']*agents_time_new_Comers_stay.loc[20,'emptyPalces'].astype(int)
dotMAtrixFig1 = exportFigure_sq(list_,('temp'),MatrixPlot)



In [288]:
dotMAtrixFig1

In [289]:

def exportFigure_sq(list_,date_,MatrixPlot):

    # Use column names of df for the different parameters x, y, color, ...
    MatrixPlot['Who'] = list_
    fig = px.scatter(MatrixPlot, x="x", y="y", color="Who",
                     title=f'Staying vs Leaving Snapshot {date_}',
                     labels={"Who":"Legend"}, # customize axis label
                     template='simple_white',color_discrete_map={
                         "Staying": "orange",
                         "New Comers": "blue",
                         "Future Units": "hsv(0,0%,95%)"})
    fig.update_layout(width=550,height=450,margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
    fig.update_yaxes(showticklabels=False,visible=False)
    fig.update_xaxes(showticklabels=False,visible=False)
    fig.update_traces(marker=dict(size=6))

    return fig

In [290]:
#fig = px.line(agents_stat_summary_by_year, x="year"})

def renters_owners_graph(year_q):
  rent_own_new_stay = agents_stat_summary_by_year[['year','New Comers_rent','New Comers_own','stay_rent','stay_own']].copy().fillna(0)
  end_index = rent_own_new_stay[rent_own_new_stay['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  rent_own_new_stayselected = rent_own_new_stay[rent_own_new_stay.index.isin(selected_indexes)]

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['New Comers_rent'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Renters",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['New Comers_own'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Owners",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['stay_rent'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Renters",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['stay_own'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Owners",
      mode="lines",
      line=dict(color='firebrick', width=5)
  ))



  fig.update_layout(title="Staying Laving and Owenrship",template='plotly_white',yaxis = {'title' : "Absolute Numbers"},margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
                    width=1100,height=350,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  # fig.update_xaxes(
  #     ticktext=list(year_ranges),
  #     tickvals=list(year_ranges)
  # )
  #fig.update_xaxes(range=list(year_ranges))
  fig.update_xaxes(range = [0,len(year_ranges)])
  fig.update_yaxes(range = [0,2100])
  return fig

# set range
renters_owners_fig = renters_owners_graph('2015 Q1')

In [291]:



def construction_typo_graph_generator(construction_typo_v,construction_typo_d):

  try:
    v_a = construction_typo_v[construction_typo_v['project_ty']==1]['count'].values[0]
  except:
    v_a = 0
  try:
    v_r = construction_typo_v[construction_typo_v['project_ty']==2]['count'].values[0]
  except:
    v_r = 0
  try:
    v_rr = construction_typo_v[construction_typo_v['project_ty']==3]['count'].values[0]
  except:
    v_rr = 0

  try:
    d_a = construction_typo_d[construction_typo_d['project_ty']==1]['count'].values[0]
  except:
    d_a = 0
  try:
    d_r = construction_typo_d[construction_typo_d['project_ty']==2]['count'].values[0]
  except:
    d_r = 0
  try:
    d_rr = construction_typo_d[construction_typo_d['project_ty']==3]['count'].values[0]
  except:
    d_rr = 0
  fig = go.Figure()

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_a,
      domain = {'x': [0.05, 0.25], 'y': [0.7, 0.9]},
      number_font_color="#2052a7",
      
      delta = {'reference': d_a, 'relative': True,'valueformat':'.2%'}))

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_r,
      domain = {'x': [0.3, 0.5], 'y': [0.7, 0.9]},
      number_font_color="#4c84cb",
      delta = {'reference': d_r, 'relative': True,'valueformat':'.2%'}))

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_rr,
      domain = {'x': [0.6, 0.80], 'y': [0.7, 0.9]},
      number_font_color="#87b1eb",
      delta = {'reference': d_rr, 'relative': True,'valueformat':'.2%'}))


  # Add images
  fig.add_layout_image(
          dict(
              source="https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/graphics/Building_Typology.jpg",
              xref="paper", yref="paper",
              x=0, y=0,
              sizex=1, sizey=1,
              xanchor="left", yanchor="bottom",
              layer="below")
  )

  fig.update_layout(template="plotly_white",margin={"r":0,"t":0,"l":0,"b":0,"pad":0},width=500,height=200)
  return fig


In [292]:
d = pd.to_datetime('2015-01-01')
d2 = d - datetime.timedelta(days=180)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)

bldgs_constr = bldgs1[bldgs1['status']=='Construction'].reset_index()

construction_typo_v = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
bldgs2 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)

bldgs_constr = bldgs2[bldgs2['status']=='Construction'].reset_index()

construction_typo_d = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
construction_typo_graph = construction_typo_graph_generator(construction_typo_v,construction_typo_d)

In [293]:
construction_typo_graph

# Dash

In [296]:

def prepare_dot_matrix(q_date_for_dot_matrix):
    # #agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

    # agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
    # agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
    # agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
    # agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

    # agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']

    #create an empty dataset with agent coutn
    num_of_agents = 3481
    MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
    #Number of Columns choose:
    numOfColumns=59
    numOfRows = 59
    # set the columns to display the column number
    MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
    #prepare the y column (row number)
    y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
    temp = []
    for i in range(0,numOfRows):
        temp +=y[i]
    y = temp
    MatrixPlot['y'] = y
    agents_stat_short = agents_stat_summary_by_year[['year','New Comers','stay']].copy().fillna(0)
    agents_stat_short['emptyPalces'] =agents_stat_short.apply(lambda p:num_of_agents-(p['stay']+p['New Comers']), axis=1)
    stay_count = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'stay'].values[0])
    new_comers = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'New Comers'].values[0])
    empty_space = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'emptyPalces'].values[0])
    list_ = ['Staying']* stay_count
    list_ = list_ + ['New Comers'] * new_comers
    list_ = list_ + ['Future Units'] * empty_space
    dotMAtrixFig = exportFigure_sq(list_,(q_date_for_dot_matrix),MatrixPlot)
    return dotMAtrixFig
    #dotMAtrixFig = dotMAtrixFig1

line_style = dict(weight=2, opacity=1, color='blue', fillOpacity=0,dashArray="10 10")

classes = ['Building before', 'Construction', 'Building after']
colorscale = ['#FFEDA0', '#FEB24C', '#FC4E2A']
style = dict(weight=2, opacity=1, color='white', fillOpacity=0.7)
colorbar = dlx.categorical_colorbar(categories=classes, colorscale=colorscale, width=300, height=30, position="bottomleft")

style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value == classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")


mapObj = dl.Map([dl.TileLayer(url=cartoUrl, maxZoom=20, attribution=attribution),dl.GeoJSON(data=statisticalStatsJson, options={'style':line_style}),dl.GeoJSON(data = simulatedBldgsJson,options = {'style':style_handle}, id='simulatedBldgs',hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="status")),colorbar],center=[32.0272,34.7444], zoom=16, style={'width': '100%', 'height': '600px'})


d = pd.to_datetime('2015-01-01')
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)
bldg_status_count = get_status_graph(bldg_status)


#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])


dashboard_page =  html.Div([
  html.Div([
    html.Div([
              #To Do Add static legend, that won't chnaged based on years and statistical stats
              html.Div(['Place2',mapObj
              
              ])
            ],style={'width': '34%', 'display': 'inline-block'}),

        html.Div([
            html.Table([
                html.Tr([html.Td(['Place1'],id='selectedDate',style={'direction':'rtl','width':'10%'}),
                    html.Td([
                                    dcc.Slider(id='years-slider',
             min=2015,
             max=2030,
             value=2015,
             marks={str(year):str(year) for year in np.arange(2015,2030,1)},
                                                    step=0.5
                                                    )
                    ],style={'direction':'rtl','width':'90%'})
                    
                ])
            ],style={'direction':'rtl','width':'100%'}),html.Div([dcc.Graph(id='time_seiries_graph',figure=renters_owners_fig)]),
            html.Div([html.Div([dcc.Graph(id='typo_count', figure=construction_typo_graph)],style={'width': '49%', 'display': 'inline-block'}),
                      html.Div([dcc.Graph(id='status_count',figure=bldg_status_count)],style={'width': '49%', 'display': 'inline-block'})
                      
                      ])




                                                    ], style={'width': '66%', 'float': 'right', 'display': 'inline-block'}),
              html.Div([
                            html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                                html.Div(['Pane 1'])],style={'width': '34%', 'display': 'inline-block'}),
                      html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                                html.Div(['Hello World'])],style={'width': '34%', 'display': 'inline-block'}),
                      html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                                html.Div([dcc.Graph(id='dot_matrix_fig',figure=dotMAtrixFig1),html.Div(id='test',children='test')])],style={'width': '33%', 'display': 'inline-block'}),


            ])
  ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'})
        #dcc.Store(id='eng_heb_data')
])
app = JupyterDash(__name__,suppress_callback_exceptions=True,prevent_initial_callbacks=True)
app.layout = html.Div([

    dashboard_page
])

@app.callback(
    Output('selectedDate', 'children'),
    Output('simulatedBldgs', 'data'),
    Output('status_count','figure'),
    Output('dot_matrix_fig','figure'),
    Output('time_seiries_graph','figure'),
    Output('typo_count','figure'),
    Input('years-slider', 'value')
    # Output('test','children'),
    #Output('dot_matrix_fig','figure'),

)
def update_output_div(input_value):
    #simulatedBldgsGdf
    date_return = f'Q3 {int(input_value)}'
    q_date_for_dot_matrix = f'{int(input_value)} Q3'
    d = pd.to_datetime(f'{int(input_value)}-07-01')
    if input_value % 1 == 0:
        d = pd.to_datetime(f'{int(input_value)}-01-01')
        date_return = f'Q1 {int(input_value)}'
        q_date_for_dot_matrix = f'{int(input_value)} Q1'
    
    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    bldgs['start_date'] = bldgs['start_date'].astype(str)
    bldgs['end_date'] = bldgs['end_date'].astype(str)
    bldgsJson = json.loads(bldgs.to_json())


    bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
    bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)
    if len(bldg_status)<3:
      if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
      if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
      if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)
    bldg_status_count = get_status_graph(bldg_status)
    
    d2 = d - datetime.timedelta(days=180)
    bldgs2 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)

    bldgs_constr = bldgs2[bldgs2['status']=='Construction'].reset_index()

    construction_typo_v = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
    construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)

    bldgs3 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)
    bldgs_constr = bldgs3[bldgs3['status']=='Construction'].reset_index()

    construction_typo_d = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
    construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
    construction_typo_graph = construction_typo_graph_generator(construction_typo_v,construction_typo_d)
    #bldg_status_count = get_status_graph(bldg_status)


    # Dot Matrix

    dotMAtrixFig1 = prepare_dot_matrix(q_date_for_dot_matrix)
    #renters Owners
    renters_owners_fig = renters_owners_graph(q_date_for_dot_matrix)
    return date_return,bldgsJson,bldg_status_count,dotMAtrixFig1,renters_owners_fig,construction_typo_graph

## To dos
## Add Tama 38 Construction Current Coutner
app.run_server(mode='external',debug=False,port=8050)

127.0.0.1 - - [27/Mar/2022 15:46:32] "GET /_shutdown_67f53e7e-5cc8-415a-a29e-35f3caca8e2f HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Mar/2022 15:46:32] "GET /_alive_67f53e7e-5cc8-415a-a29e-35f3caca8e2f HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>

In [281]:
    bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
    bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)

In [282]:
if len(bldg_status)<3:
  if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
  if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
  if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)

In [283]:
bldg_status

,status,count
0,Building before,71
1,Construction,0
2,Building after,0


In [278]:
len(bldg_status[bldg_status['status'].isin(['Building before'])])

1

# After Dash

In [83]:
agents_stat_summary_by_year.head()

,Unnamed: 0,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high,New Comers,stay,total_pop,New Comers_income_low_ratio,New Comers_income_medium_ratio,New Comers_income_high_ratio,stay_income_low_ratio,stay_income_medium_ratio,stay_income_high_ratio
0,0,2015 Q1,NaN,66.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.00,NaN,6403.0,NaN,8771.0,NaN,11667.00,NaN,318,NaN,579,NaN,470.0,NaN,427.0,NaN,0.0,NaN,897.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2015 Q3,NaN,68.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.00,NaN,6413.5,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2016 Q1,NaN,68.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,66.00,NaN,6413.5,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2016 Q3,NaN,66.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,65.25,NaN,6427.0,NaN,8833.5,NaN,11729.75,NaN,294,NaN,542,NaN,432.0,NaN,404.0,NaN,0.0,NaN,836.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2017 Q1,82.0,66.0,82.0,73.0,82.0,83.0,32.0,32.0,47.0,44.0,60.0,66.75,18838.105263,6496.0,21097.105263,8934.5,22033.947368,11806.25,11.0,282,14.0,520,0.0,405.0,11.0,397.0,14.0,0.0,25.0,802.0,827.0,0.0,0.013301,0.016929,0.489722,0.480048,0.0


In [84]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="Apartment Size  (q1-q3) vs Staying or New Comers",template='plotly_white',yaxis = {'title' : "m<sup>2</sup>"})

fig.show()

# set range

In [85]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="age (q1,q2,q3) of olderst person at household",template='plotly_white',yaxis = {'title' : "Average Age"})

fig.show()

# set range

In [86]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Icome Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Income Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Income Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="Income (q1-q3) of olderst person at household",template='plotly_white',yaxis = {'title' : "Inocme (New Israeli Shekels)"})

fig.show()

# set range

In [87]:
rent_own_new_stay = agents_stat_summary_by_year[['year','New Comers_rent','New Comers_own','stay_rent','stay_own']].copy().fillna(0)
end_index = rent_own_new_stay[rent_own_new_stay['year']=='2019 Q1'].index.values[0]
selected_indexes = range(0,end_index+1)
rent_own_new_stayselected = rent_own_new_stay[rent_own_new_stay.index.isin(selected_indexes)]

In [88]:
rent_own_new_stayselected

,year,New Comers_rent,New Comers_own,stay_rent,stay_own
0,2015 Q1,0.0,0.0,318,579
1,2015 Q3,0.0,0.0,310,565
2,2016 Q1,0.0,0.0,310,565
3,2016 Q3,0.0,0.0,294,542
4,2017 Q1,11.0,14.0,282,520
5,2017 Q3,11.0,14.0,282,520
6,2018 Q1,31.0,53.0,278,517
7,2018 Q3,69.0,134.0,278,517
8,2019 Q1,90.0,195.0,271,493


In [89]:
agents_stat_summary_by_yearæ

NameError: ignored

In [100]:
fig

In [46]:
fig.update_layout('layout_xaxis_range'=year_ranges)

SyntaxError: ignored

Their is a lot of ownown regrading how the population will look like because this is a complex matter

In [21]:

agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']


In [22]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_low_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Low Inocme",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_medium_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Medium Income",
    mode="lines",
    line=dict(color='royalblue', width=3)
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_high_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="High Income",
    mode="lines",
    line=dict(color='royalblue', width=5)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_low_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Renters",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_medium_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Medium",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_high_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="High",
    mode="lines",
    line=dict(color='firebrick', width=5)
))


fig.update_layout(title="Income by class",template='plotly_white',yaxis = {'title' : "ratio"})

fig.show()

# set range

In [23]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="New comers",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Staying",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.update_layout(title="Number of People",template='plotly_white',yaxis = {'title' : "Absolute"})

fig.show()

# set range

# Building Typology

In [ ]:
# Generate count status graph
def get_construction_typo_graph(construction_typo):
  fig = px.bar(bldg_status, x="status", y="count",width=500,height=200,template='plotly_white',title="Current Building Status",category_orders={'status':['Building before', 'Construction', 'Building after']})
  fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
  showlegend=True,

  # font=dict(
  #     size=15,
  # )
  )
  fig.update_yaxes(automargin=True)
  fig.update_yaxes(range=[0, 70])
  return fig

In [184]:
d1 = pd.to_datetime('2022-01-01')
d2 = d1 - datetime.timedelta(days=180)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d1 ) & (simulatedBldgsGdf['end_date']> d1 )].copy().reset_index(drop=True)
construction_typo_v = bldgs1['project_ty'].value_counts().to_frame().reset_index()
construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)
construction_typo_d = bldgs1['project_ty'].value_counts().to_frame().reset_index()
construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
#bldg_status_count = get_status_graph(bldg_status)

In [185]:
construction_typo

,project_ty,count
0,3.0,40
1,1.0,21
2,2.0,6


In [186]:
v_a = construction_typo[construction_typo['project_ty']==1]['count'].values[0]
v_r = construction_typo[construction_typo['project_ty']==2]['count'].values[0]
v_rr = construction_typo[construction_typo['project_ty']==2]['count'].values[0]

In [193]:
d1-d2

Timedelta('549 days 00:00:00')

In [194]:
import datetime

In [196]:
d1 - datetime.timedelta(days=180)

Timestamp('2021-07-05 00:00:00')

In [192]:
construction_typo_graph_generator(construction_typo_v,construction_typo_d)

Layout({
    'images': [{'layer': 'below',
                'sizex': 1,
                'sizey': 1,
                'source': ('https://raw.githubusercontent.' ... 'graphics/Building_Typology.jpg'),
                'x': 0,
                'xanchor': 'left',
                'xref': 'paper',
                'y': 0,
                'yanchor': 'bottom',
                'yref': 'paper'}],
    'template': '...'
})

In [123]:
bldgs1

,fid,Area,floors,height,bld_num,bld_addres,hebrew_adr,project_nu,units,project_na,project_ty,status,East,North,East_wgs84,North_wgs8,active,start_date,end_date,GlobalID,deltaDays,Area_1,temp_heigh,randH,geometry
0,27.0,380.365771,5.0,19.0,4.0,212_61,"בן גוריון 61, בת ים",502-0654715,20.0,בי/565,3.0,Building before,175632.450059,659078.548382,34.740617,32.024108,1.0,2000-01-01,2024-05-17,463d7638-5fc1-4edf-9aee-2a954dfd21fd,6592.0,530.723985,None,7,"POLYGON ((34.74048 32.02409, 34.74055 32.02421..."
1,28.0,435.489811,5.0,19.0,7.0,212_61,"בן גוריון 61, בת ים",502-0654715,24.0,בי/565,3.0,Building before,175610.345618,659030.895649,34.740385,32.023678,1.0,2000-01-01,2024-05-17,3041f060-7993-405a-9db7-9a6498059087,6592.0,607.639437,None,13,"POLYGON ((34.74024 32.02364, 34.74024 32.02364..."
2,29.0,357.512124,10.0,34.0,16.0,212_61,"בן גוריון 61 א, בת ים",502-0654715,58.0,בי/565,3.0,Building before,175629.245662,659049.904822,34.740585,32.023850,1.0,2000-01-01,2024-05-17,b711e052-2852-41a7-88e0-724024711201,6592.0,498.830520,None,6,"POLYGON ((34.74046 32.02383, 34.74046 32.02383..."
3,31.0,1018.470290,6.0,22.0,24.0,212_63,"בן גוריון 63, בת ים",502-0583948,24.0,בי/1/657,3.0,Building before,175591.840632,658999.307304,34.740191,32.023392,1.0,2000-01-01,2025-04-13,484736b6-a48e-4d68-9969-eedb81fb5319,6863.0,1421.044827,None,21,"POLYGON ((34.73998 32.02322, 34.74018 32.02361..."
4,32.0,280.157376,4.0,16.0,50.0,224_3,"ליבסקינד 3, בת ים",502-0196659,16.0,בי/475,3.0,Building before,175751.754095,659402.115257,34.741865,32.027031,1.0,2000-01-01,2022-09-29,677ca9d0-48a7-4ba1-b131-c8ee42cd7675,5920.0,390.925897,None,9,"POLYGON ((34.74178 32.02697, 34.74178 32.02700..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,221.0,0.000000,21.0,104.0,0.0,216_6n,"בלפור 6, בת ים",502-0316588,84.0,None,3.0,Construction,175925.258295,659934.217361,34.743678,32.031836,1.0,2021-02-17,2024-06-01,b349fcfd-b0bd-49dd-90e4-35b1e555fede,1200.0,753.869585,None,3,"POLYGON ((34.74375 32.03196, 34.74382 32.03176..."
63,222.0,0.000000,21.0,104.0,0.0,140_1n,"אוסקר שינדלר 1, בת ים",502-0316588,84.0,None,3.0,Construction,175895.068907,659926.154939,34.743359,32.031762,1.0,2021-02-17,2024-06-01,f39b48cc-0edb-47bd-ba47-a772d7d6946e,1200.0,630.136996,None,5,"POLYGON ((34.74343 32.03188, 34.74349 32.03169..."
64,223.0,0.000000,4.0,20.0,0.0,None,"יורדי הים 4, בת ים",502-0316588,0.0,None,3.0,Construction,175881.909237,659959.639984,34.743218,32.032064,1.0,2021-02-17,2024-06-01,bb0eee44-fadf-4499-8483-f7746a789fff,1200.0,432.493851,None,14,"POLYGON ((34.74308 32.03204, 34.74329 32.03217..."
65,225.0,0.000000,4.0,15.0,0.0,215_2n,"סוקולוב 2, בת ים",502-0147678,0.0,None,3.0,Building after,175738.622790,659306.705500,34.741731,32.026170,1.0,2021-11-24,2100-01-01,0cf7d47b-0a0a-46b8-9f1c-ae0e381335b7,28527.0,349.766608,None,13,"POLYGON ((34.74171 32.02612, 34.74169 32.02612..."
